In [14]:
import cv2, numpy as np
import sys, os
import matplotlib.pyplot as plt
import copy
from PIL import Image
import math

PI = np.pi

In [16]:
def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        dim = (int(w * r), height)

    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # resize the image
    resized = cv2.resize(image, dim, interpolation = inter)

    # return the resized image
    return resized

In [84]:
 def flat_field_correction (img):
    orig_shape = img.shape
    print (orig_shape)
    flat = cv2.medianBlur (img, 21)
    flat_mean = cv2.mean (flat)
    # out = (img.dot (flat_mean [:3]))
    # display_img (flat)
    # display_img (out)
    yuv = cv2.cvtColor (img, cv2.COLOR_BGR2YUV)
    equ = yuv.copy()

    equ[..., 0] = cv2.equalizeHist(yuv[..., 0])
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(12,12))
    clahe_out = yuv.copy()
    clahe_out[..., 0] = clahe.apply (yuv[..., 0])

    equ = cv2.cvtColor (equ, cv2.COLOR_YUV2BGR)
    clahe_out = cv2.cvtColor (clahe_out, cv2.COLOR_YUV2BGR)

    cv2.imshow ('original', img)
    cv2.waitKey(0)
    cv2.imshow ('hist eq', equ)
    cv2.waitKey(0)
    cv2.imshow ('clahe', clahe_out)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.imwrite ('images/clahe.jpg', clahe_out)
    # display_img (flat)
    # print (cv2.mean(flat))
    # C = (img * mean(F)) ./ F
    

In [85]:
img_name = '../../data/img_16.jpg'
original_img = cv2.imread (img_name, cv2.IMREAD_UNCHANGED)
original_img = image_resize(original_img, width=1536)
gray = cv2.cvtColor (original_img, cv2.COLOR_BGR2GRAY)
flat_field_correction (original_img)


(1098, 1536, 3)
